<a href="https://colab.research.google.com/github/abumarjikar/hf-practice/blob/main/Fine_Tuning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install transformers[sentencepiece]

In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AdamW,AutoModelForSequenceClassification,AutoTokenizer
import torch 

#load tokenizer
checkpoint="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sentences = ["This is my first fine tuning example.","This will cover fine tuning of existing model."]

batch = tokenizer(sentences,padding=True,truncation=True,return_tensors="pt")
print(batch)

batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer
import torch

raw_datasets = load_dataset("glue", "mrpc")

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenized_sentences = tokenizer(raw_datasets['train']['sentence1'][1],raw_datasets['train']['sentence2'][1])
#tokenized_sentences2 = tokenizer(raw_datasets['train']['sentence2'])
tokenized_sentences['token_type_ids']

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch

raw_datasets = load_dataset("glue","mrpc")

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

seperate_sentence1 = raw_datasets['train']['sentence1'][18]
seperate_sentence2 = raw_datasets['train']['sentence2'][18]
print(seperate_sentence1)
print(seperate_sentence2)
tokenizedSentence1 = tokenizer(seperate_sentence1)
tokenizedSentence2 = tokenizer(seperate_sentence2)
print("seperate 1: ",tokenizedSentence1)
print("seperate 2: ",tokenizedSentence2)

print("##############")
combined_tokenized = tokenizer(raw_datasets['train']['sentence1'][18],raw_datasets['train']['sentence2'][18])
print("Combined: ",combined_tokenized['token_type_ids'])

print(tokenizer.convert_ids_to_tokens(combined_tokenized['input_ids']))
#tokenizer.decode(combined_tokenized['input_ids'])

  0%|          | 0/3 [00:00<?, ?it/s]

" I think you 'll see a lot of job growth in the next two years , " he said , adding the growth could replace jobs lost .
" I think you 'll see a lot of job growth in the next two years , " said Mankiw .
seperate 1:  {'input_ids': [101, 107, 146, 1341, 1128, 112, 1325, 1267, 170, 1974, 1104, 2261, 3213, 1107, 1103, 1397, 1160, 1201, 117, 107, 1119, 1163, 117, 5321, 1103, 3213, 1180, 4971, 5448, 1575, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
seperate 2:  {'input_ids': [101, 107, 146, 1341, 1128, 112, 1325, 1267, 170, 1974, 1104, 2261, 3213, 1107, 1103, 1397, 1160, 1201, 117, 107, 1163, 2268, 2293, 2246, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
from transformers import AutoTokenizer

from datasets import load_dataset

raw_datasets = load_dataset("glue","mrpc")

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenized_dataset = tokenizer(
    raw_datasets['train']['sentence1'],
    raw_datasets['train']['sentence2'],
    padding=True,
    truncation=True
)

tokenized_dataset

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

def tokenizer_function(example):
  return tokenizer(example['sentence1'],example['sentence2'],truncation=True)

raw_datasets = load_dataset("glue","mrpc")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_dataset = raw_datasets.map(tokenizer_function,batched=True)

tokenized_dataset

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

def tokenizer_function(example):
  return tokenizer(example['sentence'],truncation=True)

raw_datasets = load_dataset("glue","sst2")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenized_dataset = raw_datasets.map(tokenizer_function,batched=True)
tokenized_dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from transformers import DataCollatorWithPadding



def tokenizer_function(example):
  return tokenizer(example['sentence1'],example['sentence2'], truncation=True)

raw_data = load_dataset("glue","mrpc")

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

tokenized_data = raw_data.map(tokenizer_function,batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainning_config = TrainingArguments("trained-bert-for-mrpc")

trainer = Trainer(
    model,
    trainning_config,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()